In [10]:
import polars as pl
from pyiceberg.catalog.rest import RestCatalog

pl.Config.set_thousands_separator(",")
pl.Config.set_float_precision(2)

polars.config.Config

# Querying the data

Now we have some data and we understand a bit better how Iceberg works under the hood. It's time to actually start using it!

The key selling point for Iceberg is that we have the option of using many different query engines to read from the same data storage.
To show this off, let's run some simple queries using a few different query engines. In this demo, we'll focus on locally runnable query engines, but often the pattern will be using something like Snowflake or Databricks for the heavy lifting, but have the optionality of using alternatives for different usecases.

First, we get a reference to our table, since many of these engines are using pyiceberg as a jumping-off point, either to directly interface with the Pyiceberg table, or because we can use Pyiceberg to find out the location of the current metadata.json file

In [31]:
catalog = RestCatalog(
    "lakekeeper", uri="http://lakekeeper:8181/catalog", warehouse="lakehouse"
)
table = catalog.load_table("housing.staging_prices")

## Pyiceberg

Let's see how we would use Pyiceberg directly to handle querying first. For each of these examples, we'll do something simple - we will calculate the mean monthly house price per month in 2024 (we've loaded data for 2024 and 2023 in the previous example). 

In [23]:
%%time

iceberg_results = table.scan(
    selected_fields=["price", "date_of_transfer"],
    row_filter="date_of_transfer >= '2024-01-01' and date_of_transfer <= '2024-12-31'",
)
iceberg_results.to_polars().group_by(pl.col("date_of_transfer").dt.month()).agg(
    pl.col("price").mean()
).sort(by="date_of_transfer")

CPU times: user 118 ms, sys: 91.9 ms, total: 210 ms
Wall time: 195 ms


date_of_transfer,price
i8,f64
1,"388,804.09"
2,"374,737.75"
3,"400,307.61"
4,"397,024.66"
5,"382,252.88"
…,…
8,"377,343.48"
9,"376,216.13"
10,"378,261.83"


## Polars
Pyiceberg provides us with limited filtering and projection capabilities - it provides the building blocks for libraries that build on top of Pyiceberg. We used Polars to finish the job in this example, but polars can read Iceberg directly so we can avoid the extra step

In [32]:
%%time
polars_df = (
    pl.scan_iceberg(table)
    .group_by(pl.col("date_of_transfer").dt.month())
    .agg(pl.col("price").mean())
    .sort(by="date_of_transfer")
    .collect()
)
polars_df

CPU times: user 877 ms, sys: 68.9 ms, total: 946 ms
Wall time: 2.57 s


date_of_transfer,price
i8,f64
1,"393,931.87"
2,"383,003.55"
3,"408,556.43"
4,"399,177.48"
5,"396,928.45"
…,…
8,"398,608.77"
9,"394,336.52"
10,"385,399.61"


## Duckdb
Duckdb is also an excellent choice for working with Iceberg, especially if you want to stick to SQL.

It does require some setup, since Duckdb doesn't yet know how to talk to the REST catalog, so it needs to have it's own credentials, but the [duckdb-iceberg](https://github.com/duckdb/duckdb-iceberg) extension recently got additional sponsorship from AWS to improve Iceberg compatibility, so keep an eye on that

In [13]:
import duckdb

In [14]:
# Create a duckdb connection
conn = duckdb.connect()
# Load the Iceberg extension for DuckDB
conn.install_extension("iceberg")
conn.load_extension("iceberg")


# To be able to read the Iceberg metadata, we need credentials for the bucket
conn.sql("""
CREATE OR REPLACE SECRET minio (
TYPE S3,
ENDPOINT 'minio:9000',
KEY_ID 'minio',
SECRET 'minio1234',
USE_SSL false,
URL_STYLE 'path'
)
""")

┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ true    │
└─────────┘

In [33]:
%%time
# We can read the iceberg data using DuckDB
conn.sql(f"""
SELECT month(date_of_transfer) as transfer_month, mean(price) as mean_price
FROM iceberg_scan('{table.metadata_location}')
GROUP BY 1
""").show()

┌────────────────┬────────────────────┐
│ transfer_month │     mean_price     │
│     int64      │       double       │
├────────────────┼────────────────────┤
│              1 │ 393931.86604897934 │
│              2 │  383003.5522775484 │
│              3 │  408556.4272583045 │
│              4 │ 399177.47985122725 │
│              5 │   396928.445743375 │
│              6 │ 410934.52561368525 │
│              7 │  398584.9802989558 │
│              8 │ 398608.77086901217 │
│              9 │   394336.522086173 │
│             10 │ 385399.60655165254 │
│             11 │  368753.2412621134 │
│             12 │  391226.1924619834 │
├────────────────┴────────────────────┤
│ 12 rows                   2 columns │
└─────────────────────────────────────┘

CPU times: user 159 ms, sys: 33.9 ms, total: 192 ms
Wall time: 183 ms


## Trino
Trino is another popular option, especially since AWS provides it as a serverless query engine through Athena. Trino is another SQL-based query engine, so the query looks pretty similar, just using Trino SQL dialect

In [26]:
import sqlalchemy as sa

engine = sa.create_engine("trino://trino:@trino:8080/lakekeeper")

sql = """
SELECT month(date_of_transfer) as transfer_month, avg(price) as mean_price 
FROM housing.staging_prices
GROUP BY 1
ORDER BY 1
"""

In [34]:
%%time
pl.read_database(sql, engine)

CPU times: user 15.6 ms, sys: 3.36 ms, total: 18.9 ms
Wall time: 418 ms


transfer_month,mean_price
i64,f64
1,"393,931.87"
2,"383,003.55"
3,"408,556.43"
4,"399,177.48"
5,"396,928.45"
…,…
8,"398,608.77"
9,"394,336.52"
10,"385,399.61"


## Daft
Daft is a relatively new player in the Dataframe world, similar to Polars, but also designed for scaling out. It's also written in Rust, but Daft has had early support for Iceberg - let's see if that helps

In [21]:
import daft

In [35]:
%%time
(
    daft.read_iceberg(table)
    .groupby(daft.col("date_of_transfer").dt.month())
    .agg(daft.col("price").mean())
    .sort(by=daft.col("date_of_transfer"))
    .show(12)
)

date_of_transferUInt32,priceFloat64
1,393931.86604897934
2,383003.5522775484
3,408556.4272583045
4,399177.47985122725
5,396928.445743375
...,...
8,398608.77086901217
9,394336.522086173
10,385399.60655165254
11,368753.2412621134


CPU times: user 434 ms, sys: 349 ms, total: 782 ms
Wall time: 544 ms


## Query engines
So now we've done a tour of some of the query engines that are also easy to run locally - we've been through Python with Pyiceberg, Rust with Polars and Daft, C++ with Duckdb and finally Java with Trino. One important player we've left out here is Spark. There is no denying that Iceberg was originally a Java project and the Java Iceberg libraries are the most feature-complete. 

In a real enterprise setup, you'll probably a managed service like Databricks or Snowflake that you can rely on as your main Iceberg driver - but the beauty of Iceberg is that you don't have to. You can mix and match these different query engines depending on the task at hand, while not having to move the data anywhere.

# Exercise

Try running a query using your favourite query engine to calculate the average house price for your county. If you don't live in the UK - pick the funniest sounding one. (I quite like WORCESTERSHIRE)